# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
import pandas as pd

firefoxOptions = Options()
firefoxOptions.add_argument("-headless")

navegador = webdriver.Firefox(options=firefoxOptions)

# Passo 1 - Pesquisar a cotação do dólar
# Passo 2 - Pegar cotação do dólar
navegador.get('https://www.melhorcambio.com/dolar-hoje')

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="comercial"]').get_attribute('value')
# Passo 3 - Pegar cotação do euro
navegador.get('https://www.melhorcambio.com/euro-hoje')

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="comercial"]').get_attribute('value')

                                      
# Passo 4 - Pegar cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH,
    '//*[@id="comercial"]').get_attribute('value')

navegador.quit()

cotacao_dolar = cotacao_dolar.replace(',','.')
cotacao_euro = cotacao_euro.replace(',','.')
cotacao_ouro = cotacao_ouro.replace(',','.')





### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [5]:
# Passo 5 - Importar a lista de produtos
import pandas as pd

base = pd.read_excel('recursos/Produtos.xlsx')
display(base)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [6]:
# Passo 6 - Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
# Dólar
base.loc[base["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

# Euro
base.loc[base["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

# Ouro
base.loc[base["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
base['Preço de Compra'] = base['Preço Original'] * base['Cotação']

# atualizar o preço final (preço base reais * Margem)
base['Preço de Venda'] = base['Preço de Compra'] * base['Margem']


display(base)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.06,5059.9494,1.40,7083.92916
1,Carro Renault,4500.00,Euro,5.27,23715.0000,2.00,47430.00000
2,Notebook Dell,899.99,Dólar,5.06,4553.9494,1.70,7741.71398
3,IPhone,799.00,Dólar,5.06,4042.9400,1.70,6872.99800
4,Carro Fiat,3000.00,Euro,5.27,15810.0000,1.90,30039.00000
5,Celular Xiaomi,480.48,Dólar,5.06,2431.2288,2.00,4862.45760
6,Joia 20g,20.00,Ouro,294.60,5892.0000,1.15,6775.80000


### Agora vamos exportar a nova base de preços atualizada

In [7]:
# exportação do arquivo
base.to_excel("recursos/Produtos_atualizado.xlsx", index= False)# sem índice